In [13]:
import cv2
import time
import numpy as np
from utils import *

In [14]:
img = cv2.imread("Images/rotated-board.jpg", 1)

HEIGHT, WIDTH = img.shape[:2]
AREA = HEIGHT * WIDTH

np.save("dimensions", np.array([HEIGHT, WIDTH]))


CELL_AREA_MIN = int(AREA * 0.8 / (27*27))
CELL_AREA_MAX = int(AREA / (9*9)) + 1

show(img)

False

In [4]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5, 5), 0)
# blur = cv2.bilateralFilter(gray, 15, 25, 25)
thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 9, 4)
thresh2 = cv2.erode(thresh, KERNEL5)

edged = cv2.Canny(gray, 80, 200)


show(thresh, thresh2)

False

In [5]:
contours, _ = cv2.findContours(thresh2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

cimg = draw(img, contours)
show(cimg, thresh2)

False

In [6]:
proImgg = img.copy()
proImgr = img.copy()
proImgb = img.copy()

area = []

for cnt in contours:

    cur = cv2.contourArea(cnt)
    draw_on(proImgb, [cnt], thickness=-1, color='b')
    
    
    if CELL_AREA_MIN < cur < CELL_AREA_MAX:
        
        draw_on(proImgg, [cnt], thickness=-1)
            
        simplified = approxPoly(cnt)
        
        if len(simplified) == 4:
            area.append(cur)
            draw_on(proImgr, [cnt], color='r', thickness=-1)


show(proImgg, proImgr)

area.sort()
print(len(area))

82


In [7]:
Area, _ = frameArea(area)
Area

181170.0

In [8]:
outerContour = None

blurIntense = cv2.GaussianBlur(thresh, (11, 11), 0)
thresh3 = cv2.adaptiveThreshold(blurIntense, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 9, 4)
thresh3 = cv2.erode(thresh3, KERNEL7)

show(blurIntense, thresh3)

# contours2, _ = cv2.findContours(thresh3, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
contours2 = contours

cnt = draw(img, contours2)
show(cnt)

if Area is not None:
    
    for cnt in contours2:
        
        area = cv2.contourArea(cnt)
        
        
        if Area*0.5 < area < Area*2:
            
            simplified = approxPoly(cnt, factor=0.05)
            
            if len(simplified) == 4:
                cnt = draw(img, [simplified], thickness=-1)
                show(cnt)
                
                outerContour = simplified



In [12]:
if outerContour is not None:
    
    m = np.mean(outerContour)
    pt1 = np.float32(sorted(outerContour.reshape(4, 2), key=lambda x: (x[0], x[1])))
    
    pt2 = [[0, 0], [0, OUTER_DIM], [OUTER_DIM, 0], [OUTER_DIM, OUTER_DIM]]
    if pt1[0][1] > pt1[1][1]:
        pt2[0], pt2[1] = pt2[1], pt2[0]
    
    if pt1[2][1] > pt1[3][1]:
        pt2[2], pt2[3] = pt2[3], pt2[2]
    
    pt2 = np.float32(pt2)
    
    np.save("pt1", pt1)
    
    M = cv2.getPerspectiveTransform(pt1, pt2)
    new_img = cv2.warpPerspective(thresh, M, (OUTER_DIM, OUTER_DIM))
    color_img = cv2.warpPerspective(img, M, (OUTER_DIM, OUTER_DIM))
    
    
#     _, new_img = cv2.threshold(new_img, 100, 255, cv2.THRESH_BINARY_INV)
    show(img, new_img, color_img)
    
    cv2.imwrite("tmp.png", new_img)
    cv2.imwrite("col_tmp.png", color_img)
    
else:
    print("No board detected")
